# Predicting Stock Prices from News Headlines with AI



This notebook helps you explore how **language models** can analyze real-world news headlines to predict changes in stock prices.

You’ll:
- Collect news headlines about real companies (like Starbucks or Apple)
- Use a pre-trained AI model to classify the **sentiment** (positive or negative) of these news headlines
- Combine that with stock prices
- Use a forecasting model to predict future price changes
- Visualize your results interactively



In [ ]:
pip install yfinance finvizfinance transformers pandas numpy statsmodels holidays plotly ipywidgets


In [ ]:
import yfinance as yf
from finvizfinance.quote import finvizfinance
from transformers import pipeline
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import holidays
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output



In [ ]:
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1)

This uses the yfinance and finvizfinance packages to retrive real-time news data !

In [ ]:

def get_news_data(ticker):
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()
    news_df = pd.DataFrame(news_df)

    # Drop NaN and clean whitespace
    news_df = news_df.dropna(subset=["Title"])
    news_df = news_df[news_df["Title"].str.strip() != ""]
    news_df['Title'] = news_df['Title'].str.lower()
    news_df['Date'] = pd.to_datetime(news_df['Date'])
    news_df['DateOnly'] = news_df['Date'].dt.date

    return news_df.reset_index(drop=True)




## Preview the News Data

Let’s look at what the cleaned news data looks like. We’ll start with a company like Starbucks (`SBUX`).

Each row is a headline, and we’ve cleaned it to keep only useful info like the title and date.


In [ ]:
# Example: Preview the news data for a specific company (e.g., Starbucks)
ticker = "SBUX"
news_df = get_news_data(ticker)

# Show the first 5 rows
news_df.head()


## Classifying Headline Sentiment with LLM

We now apply a pre-trained **large language model** to each headline.

It returns:
- `POSITIVE` — news that sounds good (e.g., “profits surge”)
- `NEGATIVE` — news that sounds bad (e.g., “lawsuit filed”)

We skip `NEUTRAL` news to focus on strong market signals.

This uses Hugging Face’s `pipeline()` to load a model called **DistilBERT** that’s been trained on movie reviews, but works surprisingly well on financial headlines too!


In [ ]:
# classifier = pipeline("sentiment-analysis")

def classify_sentiment(text):
    return classifier(text)[0]["label"].upper()

def apply_sentiment(news_df):
    news_df["Sentiment"] = news_df["Title"].apply(classify_sentiment)
    return news_df[news_df["Sentiment"] != "NEUTRAL"]


In [ ]:
def process_sentiment(news_df):
    grouped = news_df.groupby(["DateOnly", "Sentiment"]).size().unstack(fill_value=0)
    grouped = grouped.reindex(columns=["POSITIVE", "NEGATIVE"], fill_value=0)

    grouped["7day_avg_positive"] = grouped["POSITIVE"].rolling(window=7, min_periods=1).sum()
    grouped["7day_avg_negative"] = grouped["NEGATIVE"].rolling(window=7, min_periods=1).sum()
    grouped["7day_pct_positive"] = grouped["POSITIVE"] / (grouped["POSITIVE"] + grouped["NEGATIVE"])

    return grouped.reset_index()


In [ ]:
news_df = get_news_data("AAPL")            # Step 1: Get news data
news_df = apply_sentiment(news_df)         # Step 2: Classify sentiment
sentiment_df = process_sentiment(news_df)  # Step 3: Process sentiment stats
sentiment_df.head(5)


## Understanding the Sentiment Data Table

This table summarizes how **news sentiment** changes over time for a particular stock (e.g., AAPL), using headlines pulled from financial news.

| Column                | What it means                                                                 |
|-----------------------|------------------------------------------------------------------------------|
| `DateOnly`            | The specific date of the news                                                |
| `POSITIVE`            | Number of news headlines that had **positive** sentiment on that day        |
| `NEGATIVE`            | Number of headlines that had **negative** sentiment that day                |
| `7day_avg_positive`   | Rolling 7-day **sum** of positive headlines (used to smooth out day-to-day spikes) |
| `7day_avg_negative`   | Rolling 7-day **sum** of negative headlines                                  |
| `7day_pct_positive`   | Percentage of headlines that were positive in the last 7 days:<br>  
\[
\frac{\text{Positive}}{\text{Positive} + \text{Negative}}
\]

---

###  Why is this useful?

- If the `7day_pct_positive` is rising, the **overall tone of news** is getting more optimistic.
- If it's dropping, it could mean **public or investor concern** is growing.
- We can later **plot this** and compare it against stock price to see if sentiment influences market behavior.


In [ ]:
# downloads historical stock prices and calculates daily percentage change in closing price

def get_stock_data(ticker, start, end):
    stock = yf.download(ticker, start=start, end=end)

    # Check if columns are MultiIndex (only happens with multiple tickers)
    if isinstance(stock.columns, pd.MultiIndex):
        stock.columns = ['_'.join(col).strip() for col in stock.columns]

        # Adjust close column name accordingly
        close_col = f"Close_{ticker}"
    else:
        close_col = "Close"

    stock["Pct_Change"] = stock[close_col].pct_change() * 100
    stock.reset_index(inplace=True)
    stock["DateOnly"] = stock["Date"].dt.date
    return stock[["DateOnly", "Pct_Change"]]



In [ ]:
# print("sent_df index:", sent_df.index)
# print("stock_df index:", stock_df.index)

# print("\nSENT_DF COLUMNS:")
# print(sent_df.columns)

# print("\nSTOCK_DF COLUMNS:")
# print(stock_df.columns)

# print("\nSENT_DF HEAD:")
# print(sent_df.head())

# print("\nSTOCK_DF HEAD:")
# print(stock_df.head())


In [ ]:
# merges sentiment and stock data by date, and lags sentiment by one day to align with price changes.
def combine_data(sent_df, stock_df):
    sent_df = sent_df.reset_index(drop=True)
    stock_df = stock_df.reset_index(drop=True)
    return (
        pd.merge(sent_df, stock_df, on="DateOnly", how="inner")
          .assign(lagged_sentiment=lambda df: df["7day_pct_positive"].shift(1))
    )



In [ ]:
def compute_corr(df):
    """
    Computes correlation between lagged sentiment and stock % change.
    """
    return df[["lagged_sentiment", "Pct_Change"]].corr().iloc[0, 1]


In [ ]:
ticker = "SBUX"

news = get_news_data(ticker)
news = apply_sentiment(news)
sent_df = process_sentiment(news)

start_date = sent_df["DateOnly"].min()
end_date = sent_df["DateOnly"].max()

stock_df = get_stock_data(ticker, start_date, end_date)

combined_df = combine_data(sent_df, stock_df)

corr = compute_corr(combined_df)
print("Correlation between lagged sentiment and % stock change:", corr)


In [ ]:
def get_future_dates(start_date, num_days):
    if not isinstance(start_date, pd.Timestamp):
        start_date = pd.to_datetime(start_date)

    us_holidays = holidays.US()
    future_dates = []
    current_date = start_date + pd.Timedelta(days=1)

    while len(future_dates) < num_days:
        if current_date.weekday() < 5 and current_date.date() not in us_holidays:
            future_dates.append(current_date)
        current_date += pd.Timedelta(days=1)

    return future_dates


def fit_and_forecast(combined_df, forecast_steps=3):
    combined_df = combined_df.dropna(subset=['Pct_Change', 'lagged_sentiment'])

    endog = combined_df['Pct_Change']
    exog = combined_df['lagged_sentiment']

    model = SARIMAX(endog, exog=exog, order=(1, 1, 1))
    fit = model.fit(disp=False)

    future_dates = get_future_dates(combined_df.index[-1], forecast_steps)
    future_exog = np.tile(combined_df['lagged_sentiment'].iloc[-1], forecast_steps).reshape(-1, 1)

    forecast = fit.get_forecast(steps=forecast_steps, exog=future_exog)
    return forecast.predicted_mean, forecast.conf_int(), future_dates



In [ ]:
def create_plot(combined_df, forecast_mean, forecast_ci, forecast_index):
    combined_df = combined_df.copy()
    sentiment_std = (combined_df['lagged_sentiment'] - combined_df['lagged_sentiment'].mean()) / combined_df['lagged_sentiment'].std()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=sentiment_std,
        name='Standardized Sentiment',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df['Pct_Change'],
        name='Stock % Change',
        yaxis='y2',
        line=dict(color='green')
    ))

    fig.add_trace(go.Scatter(
        x=forecast_index,
        y=forecast_mean,
        name='Forecasted % Change',
        line=dict(color='red')
    ))

    fig.add_trace(go.Scatter(
        x=np.concatenate([forecast_index, forecast_index[::-1]]),
        y=np.concatenate([forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1][::-1]]),
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    # ✅ FIXED titlefont (use inside `title` object)
    fig.update_layout(
        title='Sentiment vs Stock Change Forecast',
        xaxis_title='Date',
        yaxis=dict(
            title=dict(text='Standardized Sentiment', font=dict(color='blue'))
        ),
        yaxis2=dict(
            title=dict(text='Stock % Change', font=dict(color='green')),
            overlaying='y',
            side='right'
        ),
        template='plotly_dark'
    )

    fig.show()


In [ ]:
combined_df['DateOnly'] = pd.to_datetime(combined_df['DateOnly'])  # convert to datetime
combined_df.set_index('DateOnly', inplace=True)  # use as index
combined_df.sort_index(inplace=True)  # ensure time order




### Plot: Sentiment vs Stock % Change Forecast

This chart shows how **news sentiment** about a company relates to its **stock price changes** over time, and how we can use this relationship to make simple predictions.

- The **blue line** shows the standardized 7-day average of positive sentiment extracted from financial news headlines. A higher value means news sentiment was more positive.
- The **green line** shows the actual daily percentage change in the company’s stock price.
- The **red line** shows our simple **forecast** of future stock movement based on past sentiment trends. The shaded red area represents uncertainty around the forecast (a 95% confidence interval).

We want to see whether the **emotions in the news** (blue) can help us **predict price changes** (green and red). If they move together, it suggests that public mood might influence investor behavior.

> This kind of plot helps us visualize correlations and test basic forecasting using real-world data like stock prices and media sentiment.


In [ ]:
forecast_mean, forecast_ci, forecast_index = fit_and_forecast(combined_df)
create_plot(combined_df, forecast_mean, forecast_ci, forecast_index)


>  **Disclaimer**: This is a simplified model. In reality, stock prices are influenced by many factors, such as interest rates, earnings reports, geopolitical events, and investor speculation. This chart only considers **one variable**: news sentiment. It should not be used for actual trading decisions.